## Define Signals Attributes and Interventions on Signals Sandbox

This notebook walks you through defining a set of attributes and interventions on top of the attributes that can be tested along with the Signals Sandbox trial.

Related resources:

1. Signals Sandbox: https://try-signals.snowplow.io/
2. Tutorial for Signals Sandbox: https://docs.snowplow.io/tutorials/signals-sandbox/start/
3. Demo app that consumes the interventions defined here and can connect to your Sandbox instance: https://snowplow-incubator.github.io/signals-sandbox-ecom-demo/

### Connect to the Sandbox

The below block reads the `SP_API_URL` and `SP_SANDBOX_TOKEN` variables either from the environment or from user data on Google Colab to set up the Snowplow Signals SDK so that it can connec to your Sandbox instance.

In [ ]:
try:
    from google.colab import userdata

    ENV_SP_API_URL = userdata.get('SP_API_URL')
    ENV_SP_SANDBOX_TOKEN = userdata.get('SP_SANDBOX_TOKEN')
except ImportError:
    import os
    from dotenv import load_dotenv
    
    load_dotenv()

    ENV_SP_API_URL = os.environ.get('SP_API_URL')
    ENV_SP_SANDBOX_TOKEN = os.environ.get('SP_SANDBOX_TOKEN')

In [ ]:
from snowplow_signals import SignalsSandbox

sp_signals = SignalsSandbox(
    api_url=ENV_SP_API_URL,
    sandbox_token=ENV_SP_SANDBOX_TOKEN,
)

### Define a new attribute

This block creates definitions for attributes that are calculated on top of the Snowplow ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
    Criterion,
    Criteria,
    EventProperty,
    EntityProperty,
)

count_product_views = Attribute(
    name="count_product_views",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="product_view",
            )
        ]
    ),
    aggregation="counter",
)

count_add_to_cart = Attribute(
    name="count_add_to_cart",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="add_to_cart",
            )
        ]
    ),
    aggregation="counter",
)

total_cart_value = Attribute(
    name="total_cart_value",
    type="double",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="add_to_cart",
            )
        ]
    ),
    property=EntityProperty(
        vendor="com.snowplowanalytics.snowplow.ecommerce",
        name="product",
        major_version=1,
        path="price",
    ),
    aggregation="sum",
)

### Wrapping the attribute in a group

All attributes need to be included in attribute groups that can be considered as "tables" of attributes.

Attribute groups are immutable and versioned.

In [ ]:
from snowplow_signals import StreamAttributeGroup, domain_userid

attribute_group = StreamAttributeGroup(
    name="ecom_attributes",
    version=1,
    attribute_key=domain_userid,
    attributes=[
        count_product_views,
        count_add_to_cart,
        total_cart_value,
    ],
    owner="user@company.com",
)

### Create a service for the attribute group

A service provides an interface for consuming apps to retrieve the attribute values.

In [ ]:
from snowplow_signals import Service

stream_service = Service(
    name="ecom_attributes",
    attribute_groups=[attribute_group],
    owner="user@company.com",
)

### Publish the attribute group and service

Pulbish the attribute group and service to start processing it.

In [ ]:
sp_signals.publish([attribute_group, stream_service])

### Retrieve attributes for the service

The below snippet allows you to retrieve attributes calculated for a given domain_userid. If you haven't tracked any events with that identifier, it will return `None` for the attribute values.

In [ ]:
stream_service.get_attributes(
    signals=sp_signals,
    attribute_key="domain_userid",
    identifier="790786f6-f250-4cf0-8bca-5f1b52c4b97f",
)

### Create interventions

Interventions are rules that trigger when user attributes meet specific criteria. They enable real-time personalization by activating experiences based on current behavior. They are pushed to the client through a SSE (server-sent events) connection.

In [ ]:
from snowplow_signals import (
    RuleIntervention,
    InterventionCriterion,
    LinkAttributeKey
)

cart_abandonment_intervention = RuleIntervention(
    name="cart_abandonment",
    description="Show banner to users who have added at least one item to cart.",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:count_add_to_cart",
        operator=">",
        value=0,
    ),
    target_attribute_keys=[
        LinkAttributeKey(name="user_id"),
        LinkAttributeKey(name="domain_userid"),
    ],
    owner="user@company.com",
)

discount_intervention = RuleIntervention(
    name="discount",
    description="Show banner to users who have viewed more than 3 products.",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:count_product_views",
        operator=">",
        value=3,
    ),
    target_attribute_keys=[
        LinkAttributeKey(name="user_id"),
        LinkAttributeKey(name="domain_userid"),
    ],
    owner="user@company.com",
)

free_shipping_intervention = RuleIntervention(
    name="free_shipping",
    description="Show banner to users who have spent at least $50.",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:total_cart_value",
        operator=">",
        value=100,
    ),
    target_attribute_keys=[
        LinkAttributeKey(name="user_id"),
        LinkAttributeKey(name="domain_userid"),
    ],
    owner="user@company.com",
)

### Publish the interventions

Publish the interventions to start processing it.

In [ ]:
sp_signals.publish(
    [
        cart_abandonment_intervention,
        discount_intervention,
        free_shipping_intervention,
    ]
)